In [ ]:
import math as m
from copy import copy
import xlrd
import csv 
import math
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import normaltest

import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPClassifier as NeuralNetwork
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score

from statsmodels.tools.tools import add_constant
from statsmodels.stats.stattools import durbin_watson
from statsmodels.regression.linear_model import OLS
from statsmodels.graphics.gofplots import qqplot
import statsmodels.stats.api as sms
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



### **Data Analysis**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd


original_data = pd.read_csv("drive/MyDrive/NLP/OnlineNewsPopularity.csv")
original_data.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505



### **Encoding target variable as a categorical variable**


In [ ]:
original_data[" shares"] = np.where(original_data[" shares"]<1400,0,1)

In [ ]:
original_data.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,0


In [ ]:
original_data.columns

Index(['url', ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
       ' LDA_03', ' LDA_04', ' global_subjectivity',
       ' global_sent

In [ ]:
X = original_data[[' n_tokens_title', ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs',
                   ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus',
                   ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max',
                   ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares', ' self_reference_avg_sharess',
                   ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday',
                   ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04', ' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words',
                   ' global_rate_negative_words', ' rate_positive_words', ' rate_negative_words', ' avg_positive_polarity', ' min_positive_polarity', ' max_positive_polarity',
                   ' avg_negative_polarity', ' min_negative_polarity', ' max_negative_polarity', ' title_subjectivity', ' title_sentiment_polarity', ' abs_title_subjectivity',
                   ' abs_title_sentiment_polarity']].copy()


y = original_data[' shares'].copy()


In [ ]:
from sklearn.model_selection import train_test_split

# Split into validation set
X_remained, X_test, y_remained, y_test = train_test_split(X, y, test_size = 0.2, random_state = 52)
X_train, X_val, y_train, y_val = train_test_split(X_remained, y_remained, test_size = 0.25, random_state = 52)


In [ ]:

from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(random_state = 39, n_jobs= -1)

model.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1, random_state=39)

In [ ]:
# Make validation predictions
test_preds = model.predict_proba(X_val)



In [ ]:

preds_df = pd.DataFrame(test_preds, columns = [0, 1])

# Convert into predictions
preds_df['prediction'] = preds_df[[0, 1]].idxmax(axis = 1)
preds_df['confidence'] = preds_df[[0, 1]].max(axis = 1)

preds_df.head()



target_names = [0, 1]



filia = open('___results____', 'a')


from sklearn.metrics import f1_score, make_scorer, accuracy_score, balanced_accuracy_score
from sklearn.metrics import classification_report, matthews_corrcoef


print('__________', file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('F1 score_macro:', round(f1_score(y_test, preds_df['prediction'], average = 'macro'), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('F1 score_micro:', round(f1_score(y_test, preds_df['prediction'], average = 'micro'), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('F1 score_weighted:', round(f1_score(y_test, preds_df['prediction'], average = 'weighted'), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('accuracy_score:', round(accuracy_score(y_test, preds_df['prediction'], normalize=True, sample_weight=None), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('balanced_accuracy_score:', round(balanced_accuracy_score(y_test, preds_df['prediction']), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)


print('classification_report:', classification_report(y_test, preds_df['prediction']), file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)


print('matthews_corrcoef:', round(matthews_corrcoef(y_test, preds_df['prediction']), 5), file = filia)


filia.close()


Important features

In [ ]:
# Feature Selection Before Hand (LightGBM)
# https://www.kaggle.com/pnprabakaran/feature-selection-before-hand-lightgbm
    
# Feature importance

import lightgbm as lgb

#lightGBM model fit
gbm = lgb.LGBMRegressor()
gbm.fit(X_train, y_train)
gbm.booster_.feature_importance()

# importance of each attribute
fea_imp_ = pd.DataFrame({'cols':X_train.columns, 'fea_imp':gbm.feature_importances_})
fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

,cols,fea_imp
24,kw_max_avg,123
25,kw_avg_avg,114
40,LDA_03,113
37,LDA_00,112
4,n_non_stop_unique_tokens,101
42,global_subjectivity,100
41,LDA_04,93
39,LDA_02,90
2,n_unique_tokens,89
26,self_reference_min_shares,89


In [ ]:
X = original_data[[' kw_max_avg']].copy()


y = original_data[' shares'].copy()


In [ ]:
from sklearn.model_selection import train_test_split

# Split into validation set
X_remained, X_test, y_remained, y_test = train_test_split(X, y, test_size = 0.2, random_state = 52)
X_train, X_val, y_train, y_val = train_test_split(X_remained, y_remained, test_size = 0.25, random_state = 52)



from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(random_state = 39, n_jobs= -1)

model.fit(X_train, y_train)



# Make validation predictions
test_preds = model.predict_proba(X_val)




preds_df = pd.DataFrame(test_preds, columns = [0, 1])

# Convert into predictions
preds_df['prediction'] = preds_df[[0, 1]].idxmax(axis = 1)
preds_df['confidence'] = preds_df[[0, 1]].max(axis = 1)

preds_df.head()



target_names = [0, 1]



filia = open('___results_selected_features_1____', 'a')


from sklearn.metrics import f1_score, make_scorer, accuracy_score, balanced_accuracy_score
from sklearn.metrics import classification_report, matthews_corrcoef


print('__________', file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('F1 score_macro:', round(f1_score(y_test, preds_df['prediction'], average = 'macro'), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('F1 score_micro:', round(f1_score(y_test, preds_df['prediction'], average = 'micro'), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('F1 score_weighted:', round(f1_score(y_test, preds_df['prediction'], average = 'weighted'), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('accuracy_score:', round(accuracy_score(y_test, preds_df['prediction'], normalize=True, sample_weight=None), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)

print('balanced_accuracy_score:', round(balanced_accuracy_score(y_test, preds_df['prediction']), 5), file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)


print('classification_report:', classification_report(y_test, preds_df['prediction']), file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)
print("*****", file = filia)


print('matthews_corrcoef:', round(matthews_corrcoef(y_test, preds_df['prediction']), 5), file = filia)


filia.close()
